In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import cluster
from Funciones_auxiliares import plot_branch,get_cluster_indexs
%matplotlib widget


In [17]:
'''Extraccion de datos'''
data = '/home/lorenzo/Principal/Noise-Neuron discriminator/Datos/PCCIBNMZc'
data = pd.read_pickle(data)

Neurons = data[data.bNoise == 0]
Neurons.reset_index(inplace = True)

In [18]:
'''Ejemplo de media neuronal'''
neuron = Neurons.loc[32]
plt.plot(neuron.Mean)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
df = Neurons.copy()
'''Stadarization of the data'''
df['Mean'] = (df['Mean'] - df.Mean.apply(lambda row:np.mean(row)))/df.Mean.apply(lambda row:np.std(row))
Mean = pd.DataFrame(np.array(df.Mean.to_list())).values
'''Threshold = sqrt(-0.5ln(alpha)*(m + n )/(m*n)'''
threshold2 = 0.05
'''Plot matrix correlation clustered'''
linkage_p = cluster.hierarchy.linkage(Mean,method = 'complete',metric = 'correlation')
fl = cluster.hierarchy.fcluster(linkage_p,threshold2,criterion = 'distance')
'''Plot Teams'''
fig = plt.figure(3,figsize=(10,10))
axes = fig.subplots(int(max(fl)/4)+1,4)
axes_flt = axes.flat
for i in range(1,max(fl)+1):
    plot_branch(i,fl,axes_flt[i-1],df)
    axes_flt[i-1].title.set_text('Team {}'.format(i))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
Cluster_selected = get_cluster_indexs(8,fl)
Cluster_selected
signals = Neurons.loc[Cluster_selected]
fig = plt.figure()
for i,signal in signals.iterrows():
    plt.plot(np.arange(len(signal.Mean)),signal.Mean)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
#signal = np.concatenate([signal.Mean for i,signal in signals.iterrows()])
signal = signals.iloc
[0].Mean
plt.figure()
plt.plot(np.arange(len(signal)),signal)

AttributeError: 'list' object has no attribute 'Mean'

In [40]:
lenght = len(signal)
r = 200
Matrix = np.zeros([len(signal),r])
eps = 1
for i in range(lenght - r):
    for j in range(r):
        dist = np.abs(signal[i] - signal[i+j])
        if dist<= eps: Matrix[i,j] = 1

TypeError: object of type '_iLocIndexer' has no len()

In [41]:
from scipy.sparse import coo_matrix

def plot_coo_matrix(m,i = 0,j = 0):
    if i == 0:i = m.shape[1]
    if j == 0:j = m.shape[0]
    if not isinstance(m, coo_matrix):
        m = coo_matrix(m)
    fig = plt.figure()
    ax = fig.add_subplot(111, facecolor='white')
    ax.plot(m.col, m.row, 's', color='black', ms=1)
    ax.set_xlim(0, i)
    ax.set_ylim(0, j)
    ax.set_aspect('equal')
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.invert_yaxis()
    ax.set_aspect('equal')

    ax.set_xlabel('i')
    ax.set_ylabel('j')

    ax.xaxis.set_label_position('top')
    ax.xaxis.tick_top()
    return ax

plot_coo_matrix(Matrix.T,i = 0,j = 0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
'''Funcion embedding'''
def signal_embedding(signal,d = 2,tau =1):
    '''Se le pasa la señal, las dimensiones y el tau. Devuelve el embedding como una serie de puntos '''
    embedding = signal[np.arange(d)*(tau+1) + np.arange(np.max(signal.shape[0] - (d-1)*(tau+1), 0)).reshape(-1,1)]
    return embedding.T

In [43]:
'''Probamos con una unica senial'''
embedding = signal_embedding(signal,3,1)

AttributeError: '_iLocIndexer' object has no attribute 'shape'

In [44]:
plt.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
x,y,z = embedding
ax.plot(x, y, z, label='parametric curve')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …